# **Technical Indicators for FAANG Securities**
---

### Import Libraries

In [2]:
import os
import requests 
import pandas_datareader as pdr

import functools
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns
import warnings

from sklearn import preprocessing 

from importlib import reload
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 227

### Import Ticker Data

In [57]:
#ticker = 'FB'
#ticker = 'AAPL'
#ticker = 'AMZN'
#ticker = 'NFLX'
ticker = 'GOOG'

In [58]:
stock_df = pdr.get_data_yahoo(ticker, '1980')
# stock_df.to_csv('data/'+csv_title+'.csv')
stock_df.to_csv('data/raw_stocks/'+ticker.upper()+'.csv')

In [59]:
stock_df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-04-01,2819.000000,2775.939941,2800.199951,2814.000000,1173600.0,2814.000000
2022-04-04,2880.875000,2816.489990,2816.489990,2872.850098,953800.0,2872.850098
2022-04-05,2871.800049,2818.870117,2867.989990,2821.260010,962800.0,2821.260010
2022-04-06,2796.969971,2728.362061,2783.229980,2743.520020,1178700.0,2743.520020
2022-04-07,2752.600098,2697.145020,2732.360107,2746.385010,652804.0,2746.385010


In [60]:
stock_df.describe()

,High,Low,Open,Close,Volume,Adj Close
count,4441.000000,4441.000000,4441.000000,4441.000000,4.441000e+03,4441.000000
mean,701.193311,687.652984,694.457649,694.591157,6.379347e+06,694.591157
std,662.519431,649.812726,656.104295,656.328405,7.767880e+06,656.328405
min,50.680038,47.800831,49.409801,49.818268,7.922000e+03,49.818268
25%,249.938171,244.832306,247.502304,247.711517,1.544000e+06,247.711517
50%,437.116577,431.407990,433.699402,434.112854,3.685161e+06,434.112854
75%,1024.089966,994.070007,1009.190002,1007.039978,8.006900e+06,1007.039978
max,3042.000000,2997.750000,3037.270020,3014.179932,8.254163e+07,3014.179932


### Check for missing data

In [61]:
print('No Missing Data') if sum(stock_df.isna().sum())==0 else stock_df.isna().sum()

No Missing Data


---
# Techincal Indicator Calculations
---
- Momentum
- Relative Strength Index (RSI)
- Moving Average Convergence/Divergence(MACD)
- Bollinger Bands

> Momentum

In [62]:
def momentum(close, days):
    m = [None for i in range(days)]
    for i in range(len(close) - days):
        end = i + days
        m.append(close[i] - days)
    return m 

> Relative Strength Index (RSI)

In [63]:
def gain(x):
    if x > 0:
        return x
    else:
        return 0 
def loss(x):
    if x < 0:
        return abs(x)
    else:
        return 0 

    
def rsi(stock):
    rsi_list = [None for i in range(14)]
    stock = stock.Change
    
    # Calculate Relative Strength (RS) for first 15 
    avg_gain = sum([i for i in stock[1:15] if i > 0])/14
    avg_loss = sum([abs(i) for i in stock[1:15] if i < 0])/14 
    rs = avg_gain/avg_loss
    # Calculate Relative Strength Index (RSI) for first 15
    rsi = 100 - (100/(1+rs))
    rsi_list.append(rsi)
    
    # Calculate Relative Strength(RS) for 15+ 
    for i in range(15, len(stock)):
        avg_gain = (avg_gain * 13 + gain(stock[i]))/14
        avg_loss = (avg_loss * 13 + loss(stock[i]))/14
        rs = avg_gain/avg_loss
        
        # Calculate Relative Strength Index (RSI) for 15+ 
        rsi = 100 - (100/(1+rs))
        rsi_list.append(rsi)
    
    return rsi_list

> Moving Average Convergence/Divergence (MACD)

In [64]:
def macd(stock):
    exp1 = stock.Close.ewm(span=12, adjust=False).mean()
    exp2 = stock.Close.ewm(span=26, adjust=False).mean()
    ma_con_div = exp1-exp2
    signal = ma_con_div.ewm(span=9, adjust=False).mean()
    return ma_con_div, signal

> Bollinger Bands

In [65]:
def bollinger(stock, window=21):
    roll_mean = stock.Close.rolling(window).mean()
    roll_std = stock.Close.rolling(window).std()
    upper_band = roll_mean + (roll_std*2)
    lower_band = roll_mean - (roll_std*2)
    return upper_band, lower_band

---
# Add Technical Indicators to Dataframe 
---

In [66]:
# gets list of all files and directories
# git add .ipynb_checkpoints directory -> causes issues 
files = os.listdir('data/raw_stocks')
stocks = {}

for file in files:
    if file.endswith('.csv'):
        name = file.lower().split(".")[0]
        stocks[name] = pd.read_csv('data/raw_stocks/'+file)
    
        # Date Feature
        stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
        stocks[name].set_index('Date', inplace = True)
        # Return Feature 
        stocks[name]['Return'] = round(stocks[name]['Close']/stocks[name]['Open']-1, 3)
        # Change Feature: Change previous day's price by absolute value
        stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
        # Volatility Feature
        stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
        # Moving Average: 7 Days
        stocks[name]['MA7'] = stocks[name].Close.rolling(window=7).mean()
        # Moving Average: 21 Days
        stocks[name]['MA21'] = stocks[name].Close.rolling(window=21).mean()
    
        # Momentum
        stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
        # Relative Strength Index (RSI)
        stocks[name]['RSI'] = rsi(stocks[name])
        # Moving Average Convergence/Divergence (MACD)
        stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
        # Bollinger Bands
        stocks[name]['Upper_Band'], stocks[name]['Lower_Band'] = bollinger(stocks[name])
    
        # Saving
        # stocks[name].dropna(inplace=True)
        stocks[name].to_csv('data/stocks/'+name+'.csv')

In [67]:
stocks[ticker.lower()].tail()

,High,Low,Open,Close,Volume,Adj Close,Return,Change,Volatility,MA7,MA21,Momentum,RSI,MACD,Signal,Upper_Band,Lower_Band
Date,,,,,,,,,,,,,,,,,
2022-04-01,2819.000000,2775.939941,2800.199951,2814.000000,1173600.0,2814.000000,0.005,21.010010,107.786030,2831.507115,2714.923340,2862.000000,56.748674,41.782056,28.611745,2936.126106,2493.720574
2022-04-04,2880.875000,2816.489990,2816.489990,2872.850098,953800.0,2872.850098,0.020,58.850098,109.047209,2838.165702,2725.895252,2849.889893,61.223005,45.432480,31.975892,2954.723702,2497.066803
2022-04-05,2871.800049,2818.870117,2867.989990,2821.260010,962800.0,2821.260010,-0.016,-51.590088,107.757359,2836.855713,2739.798584,2789.989990,55.775748,43.659301,34.312574,2953.430870,2526.166298
2022-04-06,2796.969971,2728.362061,2783.229980,2743.520020,1178700.0,2743.520020,-0.014,-77.739990,105.284293,2823.215716,2749.224772,2811.000000,48.738566,35.571036,34.564267,2943.449893,2554.999651
2022-04-07,2752.600098,2697.145020,2732.360107,2746.385010,652804.0,2746.385010,0.005,2.864990,102.864138,2806.270717,2752.513579,2869.850098,48.993976,29.057255,33.462864,2943.943788,2561.083370


In [68]:
new_df = pd.read_csv('data/stocks/'+ticker.lower()+'.csv')
print('No Missing Data') if sum(new_df.isna().sum())==0 else new_df.isna().sum()

Date           0
High           0
Low            0
Open           0
Close          0
Volume         0
Adj Close      0
Return         0
Change         0
Volatility     1
MA7            6
MA21          20
Momentum       3
RSI           14
MACD           0
Signal         0
Upper_Band    20
Lower_Band    20
dtype: int64